In [1]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import user_knn
from lenskit.algorithms import Recommender
import warnings
warnings.filterwarnings('ignore') # supress warning instead of printing them every time



In [8]:
# Get the train and test data
data_test = pd.read_csv("dataset/user_negative_ratings_test_full.csv")
data_train = pd.read_csv("dataset/user_negative_ratings_train_full.csv")
display(data_test)

,UserID,JobID,Rating
0,47,169528,1
1,47,284009,1
2,72,834662,1
3,72,1020903,1
4,203,148976,1
...,...,...,...
335046,1471635,1018169,1
335047,1471635,532088,1
335048,1472089,395893,1
335049,1472089,638187,1


In [3]:
# Compute the user list to iterate through
data_users = data_test["UserID"].drop_duplicates()

In [7]:
# Create iteration loop for evaluation

hitRate = 0.0 # counting hit rate
testUserNum = 100.0 # number of test users to compute hit rate
counter = 0

for idx, user in data_users.items():

    # keep track of how many users togo through
    counter+=1
    if (counter > testUserNum): break

    # compute test instances
    testInstances = data_test[data_test["UserID"]==user] 

    # convert test instances to training instances
    trainAddOn = data_test[data_test["UserID"]!=user] 
    df_train = pd.concat([data_train, trainAddOn], ignore_index=True)
    df_train = df_train.rename(columns={"UserID": "user", "JobID": "item", "Rating": "rating"})

    # train on data
    user_user = UserUser(15, min_nbrs=3)
    recsys = Recommender.adapt(user_user)
    recsys.fit(df_train)  

    # Create recommendations
    selected_jobs = recsys.recommend(user, 100)
    
    # check if any of the jobs is contained inside
    for idx in testInstances.index:
        job = testInstances.loc[idx, "JobID"]
        # count one hit if job is in the list
        if (selected_jobs["item"] == job).any():
            hitRate+=1
            break 
    
print(hitRate/testUserNum)


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Numba is using threading layer omp - consider TBB
found 1 potential runtime problems - see https://boi.st/lkpy-perf


0.47


Hitrate 0.52 with 10000 test users